In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 34.0 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 


In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np

In [4]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')

In [5]:
# create a model
model_lab4_ex1 = ConcreteModel()

In [6]:
N = coef.shape[1]-1 #For decision variables

In [7]:
M = coef.shape[0]-1 #For number of constraints

In [8]:
# coefficients for objective function can be obtained by accessing the coef array
obj_coef = coef[0,:-1]
#print(obj_coef.shape)
#print(obj_coef)

(6,)
[  0.65   0.75  -5.85  -8.35 -12.38  30.  ]


In [9]:
#Segregating the coefficients for constraints into a separate array
constr_coef = coef[1:,:-1]
#print(constr_coef.shape)
#print(constr_coef)

(5, 6)
[[-0.5   0.   -3.8  -0.9   0.9   0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -2.9  -1.9   0.    0.   -0.9 ]
 [ 0.    0.    0.    4.7   8.02  6.15]
 [-0.7  -1.4   7.9  -2.9   0.9   0.  ]]


In [10]:
#Segregating the RHS of constraints into a separate array
constr_rhs = coef[1:,-1]
#print(constr_rhs.shape)
#print(constr_rhs)

(5,)
[ 24.8   0.8 -71.4  95.  -47. ]


In [11]:
lower_bound = 0
upper_bound = np.inf

In [12]:
col_indices = np.arange(N)
#print(col_indices)

[0 1 2 3 4 5]


In [13]:
#declare the decision variables in the model
model_lab4_ex1.x = Var(col_indices)

In [14]:
# set of row indices: since we have M contraints, we can take row indices to be the list [0,1,2,...,M-1]
row_indices = np.arange(M)
#print(row_indices)

[0 1 2 3 4]


In [15]:
model_lab4_ex1.constraints = ConstraintList()

In [16]:
for i in row_indices:
  model_lab4_ex1.constraints.add(summation(constr_coef[i],model_lab4_ex1.x) <= constr_rhs[i])

In [17]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model_lab4_ex1.x[j].setlb(lower_bound)
  model_lab4_ex1.x[j].setub(upper_bound)

In [18]:
# add the model objective
model_lab4_ex1.objective = Objective(expr = summation(obj_coef,model_lab4_ex1.x), sense=maximize)

In [19]:
model_lab4_ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [20]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [21]:
opt_cbc = SolverFactory('cbc')

In [22]:
result = opt_cbc.solve(model_lab4_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


$ \huge{7.}$ The solver termination condition and solver status messages are:  
  
Solver status: warning  
Solver termination condition: unbounded  
  
These messages are shown by the solver as we are unable to get an optimal solution for our modeled problem. Some possible reasons could be that either the sense of the problem is not proper. The other reason could be the absence of upper bounds on one or more decision variables. 

$ \huge{8.}$ We shall now try and make appropriate adjustments.

In [23]:
# reset the sense of the model objective
model_lab4_ex1.objective.set_sense(minimize)

In [24]:
model_lab4_ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [25]:
result = opt_cbc.solve(model_lab4_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


We shall now try imposing an upper bound on all the decision variables.

In [26]:
upper_bound = 50

In [27]:
for j in col_indices:
  #model_lab4_ex1.x[j].setlb(lower_bound)
  model_lab4_ex1.x[j].setub(upper_bound)

In [28]:
result = opt_cbc.solve(model_lab4_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [29]:
# display solution
print('\nObjective = ', model_lab4_ex1.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_lab4_ex1.x[i].value)

print('\nConstraints')
model_lab4_ex1.constraints.display()


Objective =  -191.71451974999997

Decision Variables
x[ 0 ] =  0.0
x[ 1 ] =  50.0
x[ 2 ] =  10.331269
x[ 3 ] =  20.212766
x[ 4 ] =  0.0
x[ 5 ] =  0.0

Constraints
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :  -57.45031159999999 :  24.8
      2 :  None :          -20.212766 :   0.8
      3 :  None :        -164.6294111 : -71.4
      4 :  None :          95.0000002 :  95.0
      5 :  None : -46.999996299999985 : -47.0


$ \huge{9.}$ The optimal objective function value, the values of variables and the constraint activities are reported as below:  
  
Objective =  -191.71451974999997

Decision Variables
$x_0$ =  0.0 \\
$x_1$ =  50.0 \\
$x_2$ =  10.331269 \\
$x_3$ =  20.212766 \\
$x_4$ =  0.0 \\
$x_5$ =  0.0 \\

Constraint Activities: \\
Constraint 1: \\
LHS value = -57.45031159999999 \\
RHS value = 24.8 \\
As the RHS and LHS values are not equal, this constraint is inactive. \\
 \\
Constraint 2: \\
LHS value = -20.212766 \\
RHS value = 0.8 \\
As the RHS and LHS values are not equal, this constraint is inactive. \\
 \\
Constraint 3: \\
LHS value = -164.6294111 \\
RHS value = -71.4 \\
As the RHS and LHS values are not equal, this constraint is inactive. \\
 \\
Constraint 4: \\
LHS value = 95.0000002 \\
RHS value = 95.0 \\
As the RHS and LHS values are extremely close to each other, this constraint is active. \\
 \\
Constraint 5: \\
LHS value = -46.999996299999985 \\
RHS value = -47.0 \\
As the RHS and LHS values are extremely close to each other, this constraint is active. \\
 \\


So, we see that on changing the sense of the model to 'minimize' and by imposing an upper bound on the decision variables, we get an optimal solution. \\
We were able to obtain a solution in this case as the decision variables were previously unbounded from above, therefore, there may have been one or more decision variables that were not able to achieve a finite upper bound value due to the nature of the constraints. Hence, by providing an upper bound explicitly, this problem was solved and all decision variables were bounded from above, which resulted in an optimal solution.  
